# Running gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-13  

Once all necessary inputs have been processed and fishing parameters calculated, we can finally run the gridded version of DBPM.  

Note that the temporal range of DBPM inputs needs to be taken into account before running the model.

## Loading relevant libraries

In [2]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
from glob import glob
import xarray as xr
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

## Defining basic variables to run gridded DBPM

In [4]:
#Name of region and model resolution
region = 'fao-88'
model_res = '025deg'

#Defining input and output folders
base_folder = '/g/data/vf71/la6889/dbpm_inputs/west_antarctica/'
gridded_folder = os.path.join(base_folder, 'gridded_params', model_res)
out_folder = os.path.join(base_folder, 'run_fishing', model_res)
#If output folder does not exist, it will create it
os.makedirs(out_folder, exist_ok = True) 

## Loading fixed DBPM parameters 

In [3]:
depth = xr.open_zarr(glob(os.path.join(base_folder, 'gridded',
                                       model_res, '*obsclim_deptho_*'))[0])['deptho']
log10_size_bins_mat = xr.open_zarr('outputs/log10_size_bins_matrix.zarr/')['size_bins']

## Loading DBPM initialisation variables
These variables include biomass for `predators` and `detritivores`, as well as `detritus`. The timestamp in all these variables should be for one month before DBPM starts.

### Option 1: If running the model for the first time
To initialise DBPM, you wil need to load the data produced in [05_setup_gridded_DBPM](new_workflow/05_setup_gridded_DBPM.ipynb).

In [8]:
predators = xr.open_zarr(glob(
    os.path.join(gridded_folder, 'predators_*'))[0])['predators'] 
detritivores = xr.open_zarr(glob(
    os.path.join(gridded_folder, 'detritivores_*'))[0])['detritivores']
detritus = xr.open_zarr(glob(
    os.path.join(gridded_folder, 'detritus_*'))[0])['detritus']

### Option 2: If running the model at specific time step
If you have already run DBPM and would like to continue running the model from a specific time step, you will need to load data for `predators`, `detritivores`, and `detritus` for the month before DBPM is to be restarted from.  

For example, if you want to restart DBPM from `1900-01`, then the `predators`, `detritivores`, and `detritus` data should be from `1899-12` needs to be loaded. Note that these data should be in the `out_folder` as they should be have produced by a previous DBPM run.

In [7]:
#Year and month before DBPM will be restarted
init_time = '1899-12'

#Loading data
predators = xr.open_dataarray(glob(
    os.path.join(out_folder, f'predators_*_{init_time}.nc'))[0])
detritivores = xr.open_dataarray(glob(
    os.path.join(out_folder, f'detritivores_*_{init_time}.nc'))[0])
detritus = xr.open_dataarray(glob(
    os.path.join(out_folder, f'detritus_*_{init_time}.nc'))[0])

## Loading dynamic variables
These refer to all variables that change along the time dimension.

In [9]:
#Intercept plankton size spectrum
ui0 = xr.open_zarr(glob(os.path.join(gridded_folder, 'ui0_spinup*'))[0])['ui0']

#Slope plankton size spectrum
slope = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res, '*spinup_slope_*'))[0])['slope']

#Temperature effects
pel_tempeffect = xr.open_zarr(glob(
    os.path.join(gridded_folder, 'pel-temp-eff_spinup*'))[0])['pel_temp_eff']

ben_tempeffect = xr.open_zarr(glob(
    os.path.join(gridded_folder, 'ben-temp-eff_spinup*'))[0])['ben_temp_eff']

#Sinking rate
sinking_rate = xr.open_zarr(
    glob(os.path.join(base_folder, 'gridded', model_res, 
                      '*_spinup_er_*'))[0])['export_ratio']

# Loading effort
effort = xr.open_zarr(glob(os.path.join(gridded_folder, 
                                        'effort_spinup*'))[0])['effort']

### Optional: If running the model at specific time step
As above, if DBPM needs to be restarted from a specific timestep, then data for dynamic variables need to be subsetted so the first time included is the same as the timestep that DBPM is being started from.

In [7]:
subset_date = '1900-01'
ui0 = ui0.sel(time = slice(subset_date, None))
slope = slope.sel(time = slice(subset_date, None))
pel_tempeffect = pel_tempeffect.sel(time = slice(subset_date, None))
ben_tempeffect = ben_tempeffect.sel(time = slice(subset_date, None))
sinking_rate = sinking_rate.sel(time = slice(subset_date, None))

Note `effort` needs additional processing when restarting from a specific date.

In [ ]:
#Load effort for time step DBPM starts
e_start = xr.open_dataarray(glob(os.path.join(out_folder, 
                                              f'effort_*_{subset_date}.nc'))[0])
#Subset effort data from the timestep after DBPM restart 
effort = effort.sel(time = slice('1900-02', None))
#Combine both data arrays
effort = xr.concat([e_start, effort], dim = 'time')

## Running gridded DBPM
Note that this may take several hours depending on computing resources, as well as the temporal and spatial range of the modelled area.

In [13]:
uf.gridded_sizemodel(gridded_folder, predators, detritivores, detritus, pel_tempeffect, 
                     ben_tempeffect, effort, ui0, sinking_rate, slope, depth, 
                     log10_size_bins_mat, region = region, model_res = model_res, 
                     out_folder = out_folder)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/minico

## Outside time loop

### Fisheries catches per year per size class
`fishing_mort_pred` and `fishing_mort_det` are set to zero outside the sizes for each class, so there is no need to apply a mask here.

In [356]:
catch_pred = fishing_mort_pred*predators*size_bin_vals
catch_det = fishing_mort_det*detritivores*size_bin_vals